<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=00f16dd28d25461303fa6608a5f1f3bbbfca488eba42358148c54090bff882f6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-30 13:05:16
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -65.92 K (-0.47%)
Current PnL: -27.29 L (-17.63%)
CY Booked + Current PnL: -13.00 L (-8.4%)
-------------------
Total profit:  1.37 L
Total loss:  -28.66 L
-------------------
Total Booked + Current PnL: 13.98 L (10.93%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.22%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.93 L (65.78%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.46,-19.89,24.94,0.09,20135.0,-20042.0,80735.0,87.41,36.0,M-SC,3.03,253.0,-1.00,0.58,6.07,OX40N,NTT,DURABLES
52,NESTLEIND,1377.00,-0.70,12.06,8.33,21.40,24777.0,32011.0,297437.0,3.91,60.0,X-LC,5.28,12.0,1.29,2.14,19.15,XY25,NTT,FMCG
82,VOLTAS,1530.00,-1.35,6.51,12.38,19.69,25283.0,12483.0,204225.0,-3.84,47.0,X-MC,2.29,78.0,0.49,1.47,13.75,XY25,NTT,AC
50,MASFIN,398.61,-0.46,-4.75,28.13,22.05,26254.0,-4650.0,93330.0,-17.80,46.0,H-SC,6.47,164.0,-0.18,0.67,35.66,XR,ATH,FINANCE
43,ITC,452.00,-0.37,-1.69,12.69,10.78,29980.0,-4070.0,236248.0,-41.56,44.0,X-LC,2.02,5.0,-0.14,1.70,3.45,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,15.92,-5.61,55.69,46.95,65861.0,-7033.0,118264.0,-12.86,62.0,H-SC,9.78,154.0,-0.11,0.85,48.16,MH,ATH,POWER
38,INDIAMART,4810.62,2.01,-3.59,114.42,106.72,136052.0,-4430.0,118906.0,-52.34,45.0,H-SC,5.31,139.0,-0.03,0.85,18.95,AR,ATH,MISC
69,SURYODAY,216.00,1.96,-18.83,51.58,23.03,74972.0,-33721.0,145350.0,57.89,55.0,H-SC,6.88,167.0,-0.45,1.05,44.22,XR,NTT,BANKS
60,REPCOHOME,880.00,1.49,-11.94,115.55,89.82,291530.0,-34207.0,252298.0,-56.04,54.0,H-SC,5.34,160.0,-0.12,1.81,33.48,XY24,NTT,FINANCE
80,VALIANTORG,838.00,1.47,-43.39,207.64,74.15,192308.0,-70989.0,92616.0,-266.56,50.0,H-SC,21.45,149.0,-0.37,0.67,18.72,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-5.00,-62.90,169.72,0.07,86376.0,-86284.0,50893.0,1764.90,46.0,L-SC,4.55,268.0,-1.00,0.37,26.57,OX40N,NTT,JEWELLERY
44,JCHAC,2282.00,-4.24,-37.58,60.20,-0.01,48023.0,-48033.0,79772.0,16226.09,35.0,M-SC,13.03,233.0,-1.00,0.57,2.10,OX40N,NTT,AC
29,HAPPSTMNDS,1480.71,-4.10,-43.40,226.62,84.85,164381.0,-55629.0,72536.0,-32.68,28.0,H-SC,13.71,147.0,-0.34,0.52,0.00,AR,ATH,IT
27,GLAXO,3466.20,-3.62,-4.15,45.17,39.15,86282.0,-8268.0,191016.0,-23.10,28.0,X-MC,10.67,60.0,-0.10,1.37,23.65,X40,ATH,PHARMA
12,BERGEPAINT,680.00,-3.60,-6.47,29.52,21.14,62767.0,-14714.0,212625.0,-13.02,36.0,X-MC,1.43,74.0,-0.23,1.53,18.08,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.48,-0.41,82.42,81.67,77000.0,-386.0,93424.0,8416.67,67.0,L-SC,15.95,271.0,-0.01,0.67,80.92,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.78,-7.67,125.18,107.91,170649.0,-11321.0,136323.0,-25.01,36.0,M-SC,11.65,216.0,-0.07,0.98,1.65,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,-0.46,-19.89,24.94,0.09,20135.0,-20042.0,80735.0,87.41,36.0,M-SC,3.03,253.0,-1.00,0.58,6.07,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.50,-20.35,25.57,0.02,35553.0,-35518.0,139041.0,112.05,32.0,M-SC,8.97,234.0,-1.00,1.00,20.94,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-0.54,-22.16,45.79,13.47,96112.0,-59769.0,209898.0,-67.92,52.0,H-SC,6.81,158.0,-0.62,1.51,6.73,XY24,NTT,PAINTS
66,SIS,528.00,-0.77,-23.38,58.27,21.26,49569.0,-25964.0,85068.0,2007.77,52.0,H-SC,4.60,166.0,-0.52,0.61,14.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.14,8.23,59.17,72.27,108512.0,13942.0,183390.0,-6.99,58.0,M-LC,5.06,99.0,0.13,1.32,14.91,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.14,8.23,59.17,72.27,108512.0,13942.0,183390.0,-6.99,58.0,M-LC,5.06,99.0,0.13,1.32,14.91,XR,NTT,IT
6,ASIANTILES,137.00,0.48,-0.41,82.42,81.67,77000.0,-386.0,93424.0,8416.67,67.0,L-SC,15.95,271.0,-0.01,0.67,80.92,XR,NTT,CERAMICS
38,INDIAMART,4810.62,2.01,-3.59,114.42,106.72,136052.0,-4430.0,118906.0,-52.34,45.0,H-SC,5.31,139.0,-0.03,0.85,18.95,AR,ATH,MISC
25,FINCABLES,1641.55,-1.78,-7.67,125.18,107.91,170649.0,-11321.0,136323.0,-25.01,36.0,M-SC,11.65,216.0,-0.07,0.98,1.65,OX40N,ATH,CABLES
48,KPIGREEN,730.66,15.92,-5.61,55.69,46.95,65861.0,-7033.0,118264.0,-12.86,62.0,H-SC,9.78,154.0,-0.11,0.85,48.16,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.37,-22.12,61.88,26.07,94079.0,-43186.0,152035.0,-62.16,23.0,X-MC,15.50,56.0,-0.46,1.09,0.00,X40N,ATH,IT
9,BAJAJHFL,181.50,-1.01,-19.33,92.74,55.47,182002.0,-47036.0,196250.0,-28.46,26.0,X-MC,16.54,64.0,-0.26,1.41,0.56,X40N,ATH,FINANCE
27,GLAXO,3466.20,-3.62,-4.15,45.17,39.15,86282.0,-8268.0,191016.0,-23.10,28.0,X-MC,10.67,60.0,-0.10,1.37,23.65,X40,ATH,PHARMA
34,HONAUT,58357.33,-0.11,-19.07,77.08,43.32,101607.0,-31052.0,131820.0,-30.40,28.0,X-SC,6.93,90.0,-0.31,0.95,1.21,X40N,ATH,ELECTRICAL
53,PAGEIND,51605.07,-2.01,-11.69,45.06,28.11,80150.0,-23535.0,177875.0,-35.98,30.0,X-MC,15.03,55.0,-0.29,1.28,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-1.90,-19.50,55.91,25.51,83759.0,-36285.0,149810.0,-8.15,33.0,H-LC,1.40,49.0,-0.43,1.08,10.40,AR,ATH,ELECTRICAL
12,BERGEPAINT,680.0,-3.60,-6.47,29.52,21.14,62767.0,-14714.0,212625.0,-13.02,36.0,X-MC,1.43,74.0,-0.23,1.53,18.08,XY24,NTT,PAINTS
78,UNITDSPR,1644.0,-1.18,3.78,16.36,20.77,39755.0,8858.0,243002.0,-5.19,46.0,X-MC,1.61,62.0,0.22,1.75,10.96,X40N,NTT,BREWERIES
20,DABUR,735.0,0.64,-3.72,49.25,43.70,117628.0,-9230.0,238838.0,-11.62,39.0,X-MC,1.96,72.0,-0.08,1.72,10.77,XY24,BTT,FMCG
43,ITC,452.0,-0.37,-1.69,12.69,10.78,29980.0,-4070.0,236248.0,-41.56,44.0,X-LC,2.02,5.0,-0.14,1.70,3.45,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.32,-39.51,123.50,35.20,96129.0,-50833.0,77837.0,2.09,32.0,X-SC,17.34,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.57,-21.84,81.06,41.51,166854.0,-57525.0,205840.0,-6.97,31.0,X-MC,8.39,61.0,-0.34,1.48,0.00,XY25,ATH,FMCG
22,DIXON,18931.72,-1.37,-22.12,61.88,26.07,94079.0,-43186.0,152035.0,-62.16,23.0,X-MC,15.50,56.0,-0.46,1.09,0.00,X40N,ATH,IT
53,PAGEIND,51605.07,-2.01,-11.69,45.06,28.11,80150.0,-23535.0,177875.0,-35.98,30.0,X-MC,15.03,55.0,-0.29,1.28,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-1.01,-19.33,92.74,55.47,182002.0,-47036.0,196250.0,-28.46,26.0,X-MC,16.54,64.0,-0.26,1.41,0.56,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.18,-29.25,101.90,42.84,46866.0,-19014.0,45992.0,-53.72,46.0,X-SC,32.06,83.0,-0.41,0.33,4.05,XY24,NTT,MISC
59,RELAXO,1176.00,-0.50,-48.03,193.05,52.31,145645.0,-69716.0,75444.0,-44.63,41.0,X-SC,7.35,91.0,-0.48,0.54,0.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.32,-39.51,123.50,35.20,96129.0,-50833.0,77837.0,2.09,32.0,X-SC,17.34,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.11,-3.58,28.68,24.08,35107.0,-4541.0,122409.0,-11.42,40.0,X-SC,7.48,89.0,-0.13,0.88,15.34,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.11,-19.07,77.08,43.32,101607.0,-31052.0,131820.0,-30.40,28.0,X-SC,6.93,90.0,-0.31,0.95,1.21,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.32,-10.31,34.59,20.71,117339.0,-39010.0,339227.0,-22.65,57.0,X-LC,3.11,1.0,-0.33,2.44,13.35,X40,ATH,IT
41,INFY,1972.00,-0.53,11.36,20.55,34.24,71606.0,35559.0,348447.0,-13.14,58.0,X-LC,7.31,2.0,0.50,2.51,20.59,X40,NTT,IT
75,TMPV,600.00,0.67,-26.01,66.20,22.98,110649.0,-58755.0,167143.0,-23.08,52.0,X-LC,6.16,3.0,-0.53,1.20,5.13,XY24,NTT,AUTO
81,VBL,671.64,0.40,-2.10,38.58,35.67,119292.0,-6635.0,309207.0,-13.74,59.0,X-LC,3.03,4.0,-0.06,2.22,11.46,X40N,ATH,FMCG
43,ITC,452.00,-0.37,-1.69,12.69,10.78,29980.0,-4070.0,236248.0,-41.56,44.0,X-LC,2.02,5.0,-0.14,1.70,3.45,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.0,-0.73,-38.46,115.96,32.89,55093.0,-29696.0,47510.0,-686.82,46.0,L-MC,6.66,259.0,-0.54,0.34,30.84,XR,NTT,BANKS
18,COFFEEDAY,80.0,0.69,-36.07,111.58,35.27,81002.0,-40954.0,72595.0,-51.71,59.0,L-SC,7.69,270.0,-0.51,0.52,77.68,XR,NTT,HOTELS
6,ASIANTILES,137.0,0.48,-0.41,82.42,81.67,77000.0,-386.0,93424.0,8416.67,67.0,L-SC,15.95,271.0,-0.01,0.67,80.92,XR,NTT,CERAMICS
13,BSOFT,831.7,-2.44,-22.79,95.28,50.79,103478.0,-32049.0,108604.0,-1.04,51.0,H-SC,11.05,151.0,-0.31,0.78,26.45,XR,ATH,IT
69,SURYODAY,216.0,1.96,-18.83,51.58,23.03,74972.0,-33721.0,145350.0,57.89,55.0,H-SC,6.88,167.0,-0.45,1.05,44.22,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.48,-0.41,82.42,81.67,77000.0,-386.0,93424.0,8416.67,67.0,L-SC,15.95,271.0,-0.01,0.67,80.92,XR,NTT,CERAMICS
48,KPIGREEN,730.66,15.92,-5.61,55.69,46.95,65861.0,-7033.0,118264.0,-12.86,62.0,H-SC,9.78,154.0,-0.11,0.85,48.16,MH,ATH,POWER
13,BSOFT,831.70,-2.44,-22.79,95.28,50.79,103478.0,-32049.0,108604.0,-1.04,51.0,H-SC,11.05,151.0,-0.31,0.78,26.45,XR,ATH,IT
74,TITAGARH,1548.00,-0.14,-14.67,70.79,45.74,146286.0,-35529.0,206648.0,13.67,71.0,H-SC,5.16,174.0,-0.24,1.49,34.40,XY24,NTT,ENGINEERING
18,COFFEEDAY,80.00,0.69,-36.07,111.58,35.27,81002.0,-40954.0,72595.0,-51.71,59.0,L-SC,7.69,270.0,-0.51,0.52,77.68,XR,NTT,HOTELS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.44
1,25,45.02
2,50,76.62


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.67
LC    24.35
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.94
X40      23.10
X40N     12.87
XR        9.84
XY25      9.19
AR        8.78
OX40N     7.48
SR        0.99
MH        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.14
H-SC    23.98
X-LC    20.89
M-SC    11.45
X-SC     8.03
H-MC     4.80
L-SC     1.56
M-MC     1.39
M-LC     1.32
H-LC     1.08
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.44,-7.62,42.22
IT,12.86,-17.60,76.74
FINANCE,9.65,-18.65,70.26
MISC,7.15,-30.49,83.87
ELECTRICAL,5.93,-13.95,55.69
PAINTS,5.80,-14.55,31.57
INSURANCE,4.73,-2.96,38.20
PHARMA,4.05,-4.37,37.23
AUTO,2.86,-31.91,77.64


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3175403.0,21
AR,1352033.0,10
XR,1294056.0,13
X40,1011872.0,14
X40N,893745.0,9
OX40N,752894.0,10
XY25,364672.0,6
SR,282463.0,2
MH,65861.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3645174.0,25
X-MC,1468965.0,16
M-SC,1466745.0,15
X-LC,821840.0,11
X-SC,802663.0,8
H-MC,411403.0,3
L-SC,244378.0,3
M-LC,108512.0,1
H-LC,83759.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249822.0      6
           AR         920524.0      5
M-SC       XY24       826745.0      6
H-SC       XR         793076.0      7
X-MC       X40        500849.0      7
           XY24       416570.0      3
           X40N       359409.0      4
H-SC       OX40N      333428.0      4
M-SC       OX40N      333090.0      5
X-LC       X40        325714.0      5
X-SC       X40N       318276.0      3
           XY24       299078.0      3
H-SC       SR         282463.0      2
H-MC       AR         220213.0      2
X-LC       X40N       216060.0      2
X-MC       XY25       192137.0      2
X-LC       XY24       191998.0      2
H-MC       XY24       191190.0      1
X-SC       X40        185309.0      2
M-SC       XR         179373.0      2
L-SC       XR         158002.0      2
M-SC       AR         127537.0      2
M-LC       XR         108512.0      1
X-LC       XY25        88068.0      2
L-SC       OX40N       86376.0      1
H-LC       AR          83759.0      1
H-SC       MH          65861.0      1
L-MC       XR          55093.0      1
M-MC       XY25        51817.0      1
L-LC       XY25        32650.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
